In [1]:
!pip install langchain openai weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 652.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Unin

In [5]:
pip install python-dotenv

In [15]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 4.8 MB/s eta 0:00:00


In [22]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [3]:
from google.colab import userdata
OPENAI_API_KEY= userdata.get('OPENAI_API_KEY')

In [6]:
import dotenv
dotenv.load_dotenv()

False

In [82]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader=PyPDFDirectoryLoader("documents/")

In [83]:
data=loader.load()

In [84]:
data

[Document(page_content='1/23/24, 3:23 PM Large language model - Wikipedia\nhttps://en.wikipedia.org/wiki/Large_language_model 1/21\nAn illustration of main components of the\ntransformer model from the original paper,\nwhere layers were normalized after (instead of\nbefore) multiheaded attention.Large language model\nA large language model  (LLM ) is a language model  nota ble for its ability to achieve gene ral-purpose language generation.\nLLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive\nself-supervised  and semi-supervised  training process.[1] LLMs are artificial neural networks  typically built with a\ntransformer -based architecture. Some recent implementations are based on alternative architectures such as recurrent\nneural network  variants and Mamba  (a state space  model).[2][3][4]\nLLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next\

In [85]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(data)

In [86]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(api_key= OPENAI_API_KEY),
    by_text = False
)

embedded weaviate is already listening on port 8079


In [87]:
retriever = vectorstore.as_retriever()

In [88]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [111]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(api_key= OPENAI_API_KEY, model_name="gpt-3.5-turbo-16k", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "Please tell me the latest LLM models and the year of release"
print(rag_chain.invoke(query))

The latest LLM models and their release years are as follows:
1. LaMDA - January 2022
2. GPT-NeoX - February 2022
3. Chinchilla - March 2022
4. PaLM (Pathways Language Model) - April 2022
5. OPT (Open Pretrained Transformer) - May 2022
6. YaLM 100B - June 2022
7. Minerva - June 2022
8. BLOOM - July 2022
9. Galactica - November 2022
10. AlexaTM (Teacher Models) - November 2022
11. LLaMA (Large Language Model Meta AI) - February 2023
12. GPT-4 - March 2023
13. Cerebras-GPT - March 2023
14. Falcon - March 2023
15. BloombergGPT - March 2023
16. PanGu-Σ - March 2023
17. OpenAssistant - March 2023
18. Claude 2 - November 2023
19. Grok-1 - November 2023
20. Gemini - December 2023
21. Mixtral 8x7B - December 2023
22. Phi-2 - December 2023
23. Foundation models - Release dates not specified.
